In [ ]:
!pip install transformers sumy spaCy nltk PyPDF2 pdfminer.six torch spacy-llm PyMuPDF sentence_transformers rouge # Alternative PDF text extraction library
!python -m spacy download en_core_web_sm  # or a larger model if needed


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 11.5 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadabilit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize
from pdfminer.high_level import extract_text
from transformers import pipeline
import spacy
from transformers import pipeline
nltk.download('punkt')

def load_hf_ner_model():
    # Load a pre-trained NER model from Hugging Face
    nlp = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
    return nlp

def perform_ner_with_hf_model(text, hf_ner_model):
    # Apply the Hugging Face NER model to the text
    ner_results = hf_ner_model(text)
    return ner_results


def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

def preprocess_contract(text):
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)  # Remove hyphenation
    text = text.replace('\n', ' ').replace('\r', '')  # Replace line breaks
    text = re.sub(r'[^A-Za-z0-9\s.,]', '', text)  # Remove special characters
    sections = re.split(r'\n\s*\d+\.\s*[A-Za-z]+', text)  # Segment into sections
    text = ' '.join([re.sub(r'^\s*\d+\.\s*', '', section, flags=re.MULTILINE) for section in sections])  # Normalize lists
    sentences = sent_tokenize(text)  # Sentence segmentation
    return sentences

def perform_ner_on_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)  # Extract text from PDF
    cleaned_sentences = preprocess_contract(text)  # Preprocess the text

    hf_ner_model = load_hf_ner_model()  # Load Hugging Face NER model

    for sentence in cleaned_sentences:
        ner_results = perform_ner_with_hf_model(sentence, hf_ner_model)
        for entity in ner_results:
            print(f"Entity: {entity['word']}, Label: {entity['entity_group']}")

# Example usage
pdf_path = '/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf'
perform_ner_on_pdf(pdf_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


Entity: MA, Label: MISC
Entity: E, Label: ORG
Entity: ONeil Construction Co. of, Label: ORG
Entity: Arizona, Label: LOC
Entity: E, Label: LOC
Entity: Kerby Ave Phoenix, Label: LOC
Entity: A, Label: LOC
Entity: ##Z, Label: LOC
Entity: Master Subcontract Agreement, Label: MISC
Entity: Blind Ideas, Label: ORG
Entity: W, Label: LOC
Entity: Larkspur Dr, Label: LOC
Entity: P, Label: LOC
Entity: ##eoria, Label: LOC
Entity: A, Label: LOC
Entity: ##Z, Label: LOC
Entity: E, Label: ORG
Entity: ONE, Label: ORG
Entity: ARI, Label: LOC
Entity: ##ZONA, Label: ORG
Entity: General Contract, Label: ORG
Entity: Contract, Label: ORG
Entity: B. C. D. E, Label: ORG
Entity: F, Label: ORG
Entity: General Contractor, Label: ORG
Entity: Subcontract, Label: ORG
Entity: Agreement, Label: MISC
Entity: General Contract, Label: ORG
Entity: Subcon, Label: ORG
Entity: Doc, Label: MISC
Entity: General Contract, Label: ORG
Entity: ##con, Label: ORG
Entity: General Contractor, Label: ORG
Entity: Agreement, Label: MISC
En

KeyboardInterrupt: 

In [ ]:
import fitz  # PyMuPDF
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text

# Summarize using vector embeddings and a custom prompt
def summarize_with_prompt(pdf_path, custom_prompt, num_sentences=5):
    # Load and preprocess text
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)

    # Split text into sentences
    sentences = cleaned_text.split('. ')

    # Load Sentence-BERT model
    model = SentenceTransformer('all-mpnet-base-v2') # all-MiniLM-L6-v2 or all-mpnet-base-v2

    # Generate embeddings for sentences and prompt
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    prompt_embedding = model.encode(custom_prompt, convert_to_tensor=True)

    # Compute similarity of each sentence to the prompt
    similarities = util.pytorch_cos_sim(prompt_embedding, sentence_embeddings)[0]

    # Select top-n sentences with highest similarity
    top_sentence_indices = similarities.topk(num_sentences)[1]
    summary_sentences = [sentences[idx] for idx in top_sentence_indices]

    # Combine selected sentences into summary
    summary = '. '.join(summary_sentences)
    return summary

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution. The summary should cover potential challenges, efficiencies, and strategic impacts, while also addressing any penalties, termination clauses, legal considerations, and intricate details that might affect operations. Ensure that the summary captures the essence of all sections, including risk management strategies, compliance with regulations, sustainability practices, and the implications of technological integration. The final summary should distill all pages of content into a comprehensive overview, retaining all critical points and nuances."
summary = summarize_with_prompt(pdf_path, custom_prompt, num_sentences=30)
print("Summary:", summary)


Summary: b provide complete contact information for all sub-tier vendors, suppliers, and Sub-Subcontractors to the General Contractor within ten 10 days from the date of an executed Work Order, c provide a material procurement log in a form acceptable to the General Contractor within ten 10 days from the date of an executed Work Order, and shall submit this log monthly with pay application d conform to General Contractors progress schedules and all revisions or changes made thereto, e prosecute the Subcontract Work in a prompt and diligent manner in accordance with General Contractors progress schedules without delaying or hindering General Contractors work or the work of other contractors or Subcontractor, f coordinate the Subcontract Work with that of all other contractors, Subcontractors, suppliers andor materialmen of the General Contractor or Owner, in a manner that will facilitate the efficient completion of the entire work, g acknowledge Now, therefore, in consideration of the f

In [ ]:
import fitz  # PyMuPDF
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text

# Summarize using vector embeddings and a custom prompt
def summarize_with_prompt(pdf_path, custom_prompt, num_sentences=5):
    # Load and preprocess text
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)

    # Split text into sentences
    sentences = cleaned_text.split('. ')

    # Handle case where there are not enough sentences
    if len(sentences) < num_sentences:
        num_sentences = len(sentences)

    # If no valid sentences found, return an empty summary
    if not sentences or num_sentences == 0:
        return "No valid sentences found for summarization."

    # Load Sentence-BERT model
    model = SentenceTransformer('all-mpnet-base-v2')

    # Generate embeddings for sentences and prompt
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    prompt_embedding = model.encode(custom_prompt, convert_to_tensor=True)

    # Compute similarity of each sentence to the prompt
    similarities = util.pytorch_cos_sim(prompt_embedding, sentence_embeddings)[0]

    # Select top-n sentences with highest similarity
    top_sentence_indices = similarities.topk(num_sentences)[1]
    summary_sentences = [sentences[idx] for idx in top_sentence_indices]

    # Combine selected sentences into summary
    summary = '. '.join(summary_sentences)
    return summary

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution. The summary should cover potential challenges, efficiencies, and strategic impacts, while also addressing any penalties, termination clauses, legal considerations, and intricate details that might affect operations. Ensure that the summary captures the essence of all sections, including risk management strategies, compliance with regulations, sustainability practices, and the implications of technological integration. The final summary should distill all pages of content into a comprehensive overview, retaining all critical points and nuances."
summary = summarize_with_prompt(pdf_path, custom_prompt, num_sentences=105)
print("Summary:", summary)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
# Latest working model that works with custom word embeddings -- 30th August


import fitz  # PyMuPDF
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text

# Summarize using vector embeddings and a custom prompt
def summarize_with_prompt(pdf_path, custom_prompt, num_sentences=5, output_file="summary.txt"):
    # Load and preprocess text
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)

    # Split text into sentences
    sentences = cleaned_text.split('. ')

    # Handle case where there are not enough sentences
    if len(sentences) < num_sentences:
        num_sentences = len(sentences)

    # If no valid sentences found, return an empty summary
    if not sentences or num_sentences == 0:
        return "No valid sentences found for summarization."

    # Load Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L12-v2') # all-MiniLM-L6-v2 or all-mpnet-base-v2 or all-MiniLM-L12-v2

    # Generate embeddings for sentences and prompt
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    prompt_embedding = model.encode(custom_prompt, convert_to_tensor=True)

    # Compute similarity of each sentence to the prompt
    similarities = util.pytorch_cos_sim(prompt_embedding, sentence_embeddings)[0]

    # Select top-n sentences with highest similarity
    top_sentence_indices = similarities.topk(num_sentences)[1]
    summary_sentences = [sentences[idx] for idx in top_sentence_indices]

    # Combine selected sentences into summary
    summary = '. '.join(summary_sentences)

    # Save the summary to a text file
    with open(output_file, "w") as file:
        file.write(summary)

    return summary

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution. The summary should cover potential challenges, efficiencies, and strategic impacts, while also addressing any penalties, termination clauses, legal considerations, and intricate details that might affect operations. Ensure that the summary captures the essence of all sections, including risk management strategies, compliance with regulations, sustainability practices, and the implications of technological integration. The final summary should distill all pages of content into a comprehensive overview, retaining all critical points and nuances."
output_file = "/content/new_contract_summary_miniLLM_L6.txt"
print("/content/summary_miniLLM_L12.txt")
summary = summarize_with_prompt(pdf_path, custom_prompt, num_sentences=25, output_file=output_file)
print("Summary saved to:", output_file)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
import fitz  # PyMuPDF for extracting text from PDF
import spacy  # SpaCy for NER and Relation Extraction
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Step 1: Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Step 2: Apply NER to Identify Key Entities
def perform_ner(text):
    nlp = spacy.load("en_core_web_sm")  # Load pre-trained NER model
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Step 3: Relation Extraction (simple dependency parsing for RE)
def perform_relation_extraction(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    relations = []
    for sent in doc.sents:
        for token in sent:
            if token.dep_ in ("nsubj", "dobj", "pobj", "prep"):  # Simple RE rules
                relations.append((token.text, token.dep_, token.head.text))
    return relations

# Step 4: Use T5 Model for Summarization
def generate_summary(text, custom_prompt, model_name="t5-base", max_length=500):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_text = custom_prompt + ":" + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=4096, truncation=False)

    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=3.0, num_beams=10, early_stopping=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main Function to Perform End-to-End Process
def summarize_pdf(pdf_path):
    # Extract text from the PDF
    text = extract_text_from_pdf(pdf_path)
    custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution, while highlighting challenges, efficiencies, and strategic impacts."
    print("Extracted Text:", text[:1000])  # Print first 1000 characters for reference

    # Perform NER
    entities = perform_ner(text)
    print("\nNamed Entities:", entities)

    # Perform Relation Extraction
    relations = perform_relation_extraction(text)
    print("\nExtracted Relations:", relations)

    # Generate Summary
    summary = generate_summary(text,custom_prompt)
    print("\nGenerated Summary:", summary)


# Test the pipeline with a sample PDF
pdf_path = "/content/test.pdf"
summarize_pdf(pdf_path)


Extracted Text: AGREEMENT 
 
This Agreement (“Agreement) is made and entered into on this 1st of Dec,  2014 and 
shall be effective from 1st of Dec, 2015 
 
Reverse Logistics Company Private Limited, a private limited company having its 
registered office at Khasra No. 337, Chaudhary Satbir Complex, Sultanpur, MG Road, 
New Delhi 110030  (Hereinafter referred to as, “GreenDust” which expression shall, 
unless repugnant to the context or meaning thereof, be deemed to include its 
successors and assigns) of the First Part; and 
 
Jasper Infotech Pvt. Ltd. having its registered address at 246, Ist Floor, Phase-III, Okhla 
Industrial Area, New Delhi 110020(Hereinafter referred to as, “Snapdeal” which 
expression shall, unless repugnant to the context or meaning thereof, be deemed to 
include its successors and assigns) of the Second Part 
 
GREENDUST and Snapdeal are collectively referred to as the Parties and individually as 
the Party wherever the context so requires. 
 
WHEREAS: 
 
a) G

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Named Entities: [('this 1st', 'DATE'), ('2014', 'DATE'), ('1st', 'ORDINAL'), ('Dec, 2015', 'DATE'), ('Reverse Logistics Company Private Limited', 'ORG'), ('Khasra No.', 'ORG'), ('337', 'CARDINAL'), ('Chaudhary Satbir Complex', 'PERSON'), ('Sultanpur', 'GPE'), ('MG Road', 'FAC'), ('New Delhi', 'GPE'), ('110030', 'DATE'), ('Hereinafter', 'ORG'), ('First', 'ORDINAL'), ('Jasper Infotech Pvt. Ltd.', 'ORG'), ('246', 'CARDINAL'), ('Ist Floor', 'ORG'), ('Okhla \nIndustrial Area', 'PERSON'), ('New Delhi', 'GPE'), ('Second', 'ORDINAL'), ('Party', 'ORG'), ('seconds', 'TIME'), ('1', 'CARDINAL'), ('1.1', 'CARDINAL'), ('1', 'CARDINAL'), ('1.2', 'CARDINAL'), ('1', 'CARDINAL'), ('GreenDust', 'ORG'), ('1', 'CARDINAL'), ('Snapdeal', 'NORP'), ('GreenDust', 'ORG'), ('GreenDust', 'ORG'), ('2', 'CARDINAL'), ('2.1', 'CARDINAL'), ('1', 'CARDINAL'), ('20%', 'PERCENT'), ('PO', 'ORG'), ('SnapDeal', 'PRODUCT'), ('GreenDust', 'ORG'), ('3 months', 'DATE'), ('3', 'CARDINAL'), ('the “Receiving Party', 'ORG'), ('thir

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input Text: Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution, while highlighting challenges, efficiencies, and strategic impacts.:AGREEMENT 
 
This Agreement (“Agreement) is made and entered into on this 1st of Dec,  2014 and 
shall be effective from 1st of Dec, 2015 
 
Reverse Logistics Company Private Limited, a private limited company having its 
registered office at Khasra No. 337, Chaudhary Satbir Complex, Sultanpur, MG Road, 
New Delhi 110030  (Hereinafter referred to as, “GreenDust” which expression shall, 
unless repugnant to the context or meaning thereof, be deemed to include its 
successors and assigns) of the First Part; and 
 
Jasper Infotech Pvt. Ltd. having its registered address at 246, Ist Floor, Phase-III, Okhla 
Industrial Area, New Delhi 110020(Hereinafter referred to as, “Snapdeal” which 
expression shall, unless repugnant to the context or meaning th

In [ ]:
import fitz  # PyMuPDF
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text

# Summarize using T5 with a custom prompt
def summarize_with_t5(pdf_path, custom_prompt, max_length=150, output_file="summary.txt"):
    # Load and preprocess text
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)

    # Load T5 model and tokenizer
    model_name = "t5-large"  # You can also try "t5-base" or "t5-large"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Prepare input with the custom prompt
    input_text = f"summarize: {custom_prompt} {cleaned_text}"

    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=4096, truncation=False)

    # Generate the summary
    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=8, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Save the summary to a text file
    with open(output_file, "w") as file:
        file.write(summary)

    return summary

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution. The summary should cover potential challenges, efficiencies, and strategic impacts, while also addressing any penalties, termination clauses, legal considerations, and intricate details that might affect operations. Ensure that the summary captures the essence of all sections, including risk management strategies, compliance with regulations, sustainability practices, and the implications of technological integration. The final summary should distill all pages of content into a comprehensive overview, retaining all critical points and nuances."
summary = summarize_with_t5(pdf_path, custom_prompt)
print(summary)


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

subcontractor agrees to comply with all federal, state, and local laws and regulations. if subcontractor fails to comply with such requirements, he will be deemed liable. if subcontractor fails to comply, he will be deemed liable. if subcontractor fails to comply, he will be deemed liable. if subcontractor fails to comply, he will be deemed liable.


In [ ]:
# For text cleaning

import re
import fitz  # PyMuPDF

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
text = extract_text_from_pdf(pdf_path)
cleaned_text = preprocess_text(text)
print(cleaned_text)


MASTER SUBCONTRACT AGREEMENT Page 1 of PMSC07_110 Rev. Aug 2017 15 A. B. C. D. E. F. 1. 2. 3. 4. 5. 1. 1. 2. That General Contractor and Subcontractor enter into this Agreement to facilitate future projects in which General Contractor may choose to retain Subcontractor. The scope of work and other project specific terms and conditions shall be set forth in a work order Work Order for each specific project. The Subcontract Work or Work shall include all labor, equipment, materials and services to be performed by Subcontractor, and all other obligations of Subcontractor required by the Subcontract Documents, as defined below. That General Contractor may retain Subcontractor from time to time on various constructions projects, as determined by General Contractor in its sole discretion pursuant to Work Orders to be executed in connection with each such project. That this Agreement shall apply to all future Work to be provided by Subcontractor on any project pursuant to such Work Orders, pr

In [ ]:
from sentence_transformers import SentenceTransformer, util
import re
from transformers import pipeline

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Preprocess the text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:!?-]', '', text)  # Remove special characters
    return text


def perform_zero_shot_ner(text, labels):
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    entities = []
    for sentence in text.split('. '):
        result = classifier(sentence, labels)
        for i, label in enumerate(result['labels']):
            if result['scores'][i] > 0.5:  # Adjust threshold as needed
                entities.append((sentence, label))
    return entities



def filter_sentences_by_tags(entities, tags):
    relevant_sentences = []
    for sentence, label in entities:
        if label in tags:
            relevant_sentences.append(sentence)
    return relevant_sentences


def summarize_sentences(sentences, custom_prompt):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    prompt_embedding = model.encode(custom_prompt, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(prompt_embedding, sentence_embeddings)[0]
    top_sentences = [sentences[i] for i in similarities.topk(len(sentences))[1]]
    summary = '. '.join(top_sentences)
    return summary

def summarize_supply_chain_document(pdf_path, tags, custom_prompt):
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)

    entities = perform_zero_shot_ner(cleaned_text, tags)
    relevant_sentences = filter_sentences_by_tags(entities, tags)
    summary = summarize_sentences(relevant_sentences, custom_prompt)

    return summary

# Example usage
pdf_path = "/content/drive/MyDrive/SPC_Assist/Contracts/New_contract.pdf"
tags = ["Conditions", "Liabilities", "Disputes", "Termination", "Insurance", "Obligations"]
custom_prompt = "Summarize the key elements and critical processes involved in the supply chain, including procurement, production, logistics, inventory management, and distribution. The summary should cover potential challenges, efficiencies, and strategic impacts, while also addressing any penalties, termination clauses, legal considerations, and intricate details that might affect operations. Ensure that the summary captures the essence of all sections, including risk management strategies, compliance with regulations, sustainability practices, and the implications of technological integration. The final summary should distill all pages of content into a comprehensive overview, retaining all critical points and nuances."
summary = summarize_supply_chain_document(pdf_path, tags, custom_prompt)
print(summary)



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from collections import Counter

# Download necessary NLTK data
nltk.download('punkt')

def calculate_rouge(reference, candidate):
    def get_ngrams(text, n):
        tokens = word_tokenize(text.lower())
        return Counter(ngrams(tokens, n))

    def rouge_n(reference, candidate, n):
        ref_ngrams = get_ngrams(reference, n)
        cand_ngrams = get_ngrams(candidate, n)

        overlap = sum((ref_ngrams & cand_ngrams).values())
        ref_count = sum(ref_ngrams.values())
        cand_count = sum(cand_ngrams.values())

        precision = overlap / cand_count if cand_count > 0 else 0
        recall = overlap / ref_count if ref_count > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        return {'precision': precision, 'recall': recall, 'f1': f1}

    def lcs(X, Y):
        m, n = len(X), len(Y)
        L = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i-1] == Y[j-1]:
                    L[i][j] = L[i-1][j-1] + 1
                else:
                    L[i][j] = max(L[i-1][j], L[i][j-1])
        return L[m][n]

    def rouge_l(reference, candidate):
        ref_sents = sent_tokenize(reference.lower())
        cand_sents = sent_tokenize(candidate.lower())

        lcs_sum = sum(lcs(word_tokenize(r), word_tokenize(c))
                      for r in ref_sents for c in cand_sents)

        ref_words = sum(len(word_tokenize(sent)) for sent in ref_sents)
        cand_words = sum(len(word_tokenize(sent)) for sent in cand_sents)

        precision = lcs_sum / cand_words if cand_words > 0 else 0
        recall = lcs_sum / ref_words if ref_words > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        return {'precision': precision, 'recall': recall, 'f1': f1}

    rouge_1 = rouge_n(reference, candidate, 1)
    rouge_2 = rouge_n(reference, candidate, 2)
    rouge_l = rouge_l(reference, candidate)

    return {
        'ROUGE-1': rouge_1,
        'ROUGE-2': rouge_2,
        'ROUGE-L': rouge_l
    }

# Example usage
reference = cleaned_text
candidate = summary

scores = calculate_rouge(reference, candidate)

for metric, values in scores.items():
    print(f"{metric}:")
    for key, value in values.items():
        print(f"  {key}: {value:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: name 'cleaned_text' is not defined